In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
#path= '/content/drive/My Drive/bases_mentoria/primer trimestre 2019'
path= '/content/drive/My Drive/Mentoria/primer trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()


Mounted at /content/drive/


,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO
0,01/01/2019 01:32:43 AM,01/01/2019 01:00:11 AM,4992639,Cor 3 Rojo,L35,Ida
1,01/01/2019 02:26:11 AM,01/01/2019 01:00:11 AM,5489022,Cor 3 Rojo,L35,Vuelta
2,01/01/2019 02:26:14 AM,01/01/2019 01:00:11 AM,5489022,Cor 3 Rojo,L35,Vuelta
3,01/01/2019 02:32:27 AM,01/01/2019 01:59:31 AM,5317539,Cor 1 Naranja,L12,Vuelta
4,01/01/2019 02:57:48 AM,01/01/2019 01:59:31 AM,4451141,Cor 1 Naranja,L12,Vuelta


In [2]:
import datetime as dt

In [3]:
# VARIABLE FECHA DE STRING A DATETIME##
big_frame['Fecha2'] = pd.to_datetime(big_frame['FECHA'], format='%d/%m/%Y %I:%M:%S %p')

In [4]:
##OBTENEMOS HORA DEL DÍA##
big_frame['hora'] = big_frame['Fecha2'].dt.hour

In [5]:
## OBTENEMOS DIA DE LA SEMANA##
big_frame['dia_semana']=big_frame['Fecha2'].dt.day_name()

In [6]:
## OBTENEMOS MES DEL AÑO###
big_frame['MES']=big_frame['Fecha2'].dt.month

In [7]:
big_frame['cant_mes_t1'] = big_frame.groupby('TARJETA')['MES'].transform('nunique')

In [ ]:
big_frame.cant_meses.value_counts()

3    9601507
2     558665
1     129878
Name: cant_meses, dtype: int64

In [8]:
##GENERAMOS UNA VARIABLE QUE INDIQUE: LUNES A VIERNES##
mask=((big_frame.dia_semana=="Monday") | (big_frame.dia_semana=="Tuesday") | (big_frame.dia_semana=="Wednesday")  | (big_frame.dia_semana=="Thursday")   | (big_frame.dia_semana=="Friday") )  
big_frame['LaV']=0
big_frame.loc[mask, 'LaV']=1


In [ ]:
big_frame.LaV.value_counts()

1    8847218
0    1442832
Name: LaV, dtype: int64

In [9]:
## GENERAMOS UNA VARIABLE QUE INDIQUE: SABADOS Y DOMINGOS##
mask=((big_frame.dia_semana=="Saturday") | (big_frame.dia_semana=="Sunday"))
big_frame['Sabados_Domingos']=0
big_frame.loc[mask, 'Sabados_Domingos']=1

In [ ]:
big_frame.Sabados_Domingos.value_counts()

0    8847218
1    1442832
Name: Sabados_Domingos, dtype: int64

In [10]:
## GENERAMOS UN INDICADOR DE HORA DE USO##
## PICO / NO PICO##
## 7 ,8, 17, 18, 13, 14 HORAS DE MAYOR USO##
mask=(((big_frame.hora==7)| (big_frame.hora==8) | (big_frame.hora==17) | (big_frame.hora==18) | (big_frame.hora==13) | (big_frame.hora==14)) & (big_frame.LaV==1))
big_frame['PICO']=0
big_frame.loc[mask, 'PICO']=1

In [ ]:
big_frame.PICO.value_counts()

0    6257108
1    4032942
Name: PICO, dtype: int64

In [ ]:
big_frame.head(4)

,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO,Fecha2,hora,dia_semana,MES,cant_meses,LaV,Sabados_Domingos,PICO
0,01/01/2019 01:32:43 AM,01/01/2019 01:00:11 AM,4992639,Cor 3 Rojo,L35,Ida,2019-01-01 01:32:43,1,Tuesday,1,3,1,0,0
1,01/01/2019 02:26:11 AM,01/01/2019 01:00:11 AM,5489022,Cor 3 Rojo,L35,Vuelta,2019-01-01 02:26:11,2,Tuesday,1,3,1,0,0
2,01/01/2019 02:26:14 AM,01/01/2019 01:00:11 AM,5489022,Cor 3 Rojo,L35,Vuelta,2019-01-01 02:26:14,2,Tuesday,1,3,1,0,0
3,01/01/2019 02:32:27 AM,01/01/2019 01:59:31 AM,5317539,Cor 1 Naranja,L12,Vuelta,2019-01-01 02:32:27,2,Tuesday,1,3,1,0,0


In [ ]:
big_frame.CORREDOR.value_counts()

Cor 1 Naranja       1838082
Cor 6 Naranja       1573429
Cor 2 Azul          1418251
Cor 3 Rojo          1198971
Cor 7 Azul          1068827
Cor 5 Amarillo       870022
Cor 8 Rojo           835873
Cor 4 Amarillo       830563
TROLE                507721
Lín Anu Amarillo     148311
Name: CORREDOR, dtype: int64

In [ ]:
## TOMARIA LAS SIGUIENTES VARIABLES PARA CONTAR POR TARJETA: %LaV %Sabados_Domingos %horaPICO %por CORREDOR
## OJO, ELIMINAR TARJETAS PRESTADAS ANTES
big_frame.shape


(10290050, 14)

In [11]:
#Se depura la base, nos quedamos solo con el primer uso de la tarjeta en ese mismo viaje.
df1 = big_frame.groupby(['TARJETA','LINEA','SENTIDO','FECHAAPERTURA']).first().reset_index()

In [12]:
##PORCENTAJE DE USO POR HORA PICO NO PICO POR TARJETA##
df1['CUENTA']=1
df1['VIAJES']=df1.groupby('TARJETA')['CUENTA'].transform('sum')



In [13]:
df1['AUX']=df1.groupby('TARJETA')['PICO'].transform('sum')
df1['porc_pico']=df1.AUX/ df1.VIAJES
df1.head(5)



,TARJETA,LINEA,SENTIDO,FECHAAPERTURA,FECHA,CORREDOR,Fecha2,hora,dia_semana,MES,cant_mes_t1,LaV,Sabados_Domingos,PICO,CUENTA,VIAJES,AUX,porc_pico
0,2016806,A,Vuelta,18/03/2019 12:07:10 PM,18/03/2019 12:46:33 PM,TROLE,2019-03-18 12:46:33,12,Monday,3,3,1,0,0,1,44,32,0.727273
1,2016806,A,Vuelta,30/01/2019 07:11:42 AM,30/01/2019 07:51:14 AM,TROLE,2019-01-30 07:51:14,7,Wednesday,1,3,1,0,1,1,44,32,0.727273
2,2016806,C,Vuelta,28/03/2019 05:58:35 PM,28/03/2019 07:38:14 PM,TROLE,2019-03-28 19:38:14,19,Thursday,3,3,1,0,0,1,44,32,0.727273
3,2016806,L30,Ida,22/03/2019 08:42:18 AM,22/03/2019 08:59:17 AM,Cor 3 Rojo,2019-03-22 08:59:17,8,Friday,3,3,1,0,1,1,44,32,0.727273
4,2016806,L30,Ida,25/03/2019 05:50:03 AM,25/03/2019 06:19:20 AM,Cor 3 Rojo,2019-03-25 06:19:20,6,Monday,3,3,1,0,0,1,44,32,0.727273


In [14]:
##PORCENTAJE DE USO DE LUNES A VIERNES##
df1['AUX']=df1.groupby('TARJETA')['LaV'].transform('sum')
df1['porc_LaV']=df1.AUX/ df1.VIAJES
df1.head(5)

,TARJETA,LINEA,SENTIDO,FECHAAPERTURA,FECHA,CORREDOR,Fecha2,hora,dia_semana,MES,cant_mes_t1,LaV,Sabados_Domingos,PICO,CUENTA,VIAJES,AUX,porc_pico,porc_LaV
0,2016806,A,Vuelta,18/03/2019 12:07:10 PM,18/03/2019 12:46:33 PM,TROLE,2019-03-18 12:46:33,12,Monday,3,3,1,0,0,1,44,43,0.727273,0.977273
1,2016806,A,Vuelta,30/01/2019 07:11:42 AM,30/01/2019 07:51:14 AM,TROLE,2019-01-30 07:51:14,7,Wednesday,1,3,1,0,1,1,44,43,0.727273,0.977273
2,2016806,C,Vuelta,28/03/2019 05:58:35 PM,28/03/2019 07:38:14 PM,TROLE,2019-03-28 19:38:14,19,Thursday,3,3,1,0,0,1,44,43,0.727273,0.977273
3,2016806,L30,Ida,22/03/2019 08:42:18 AM,22/03/2019 08:59:17 AM,Cor 3 Rojo,2019-03-22 08:59:17,8,Friday,3,3,1,0,1,1,44,43,0.727273,0.977273
4,2016806,L30,Ida,25/03/2019 05:50:03 AM,25/03/2019 06:19:20 AM,Cor 3 Rojo,2019-03-25 06:19:20,6,Monday,3,3,1,0,0,1,44,43,0.727273,0.977273


In [15]:
##PORCENTAJE DE USO EN SABADOS Y DOMINGOS##
df1['AUX']=df1.groupby('TARJETA')['Sabados_Domingos'].transform('sum')
df1['porc_Sab_Dom']=df1.AUX/ df1.VIAJES
df1.head(5)

,TARJETA,LINEA,SENTIDO,FECHAAPERTURA,FECHA,CORREDOR,Fecha2,hora,dia_semana,MES,cant_mes_t1,LaV,Sabados_Domingos,PICO,CUENTA,VIAJES,AUX,porc_pico,porc_LaV,porc_Sab_Dom
0,2016806,A,Vuelta,18/03/2019 12:07:10 PM,18/03/2019 12:46:33 PM,TROLE,2019-03-18 12:46:33,12,Monday,3,3,1,0,0,1,44,1,0.727273,0.977273,0.022727
1,2016806,A,Vuelta,30/01/2019 07:11:42 AM,30/01/2019 07:51:14 AM,TROLE,2019-01-30 07:51:14,7,Wednesday,1,3,1,0,1,1,44,1,0.727273,0.977273,0.022727
2,2016806,C,Vuelta,28/03/2019 05:58:35 PM,28/03/2019 07:38:14 PM,TROLE,2019-03-28 19:38:14,19,Thursday,3,3,1,0,0,1,44,1,0.727273,0.977273,0.022727
3,2016806,L30,Ida,22/03/2019 08:42:18 AM,22/03/2019 08:59:17 AM,Cor 3 Rojo,2019-03-22 08:59:17,8,Friday,3,3,1,0,1,1,44,1,0.727273,0.977273,0.022727
4,2016806,L30,Ida,25/03/2019 05:50:03 AM,25/03/2019 06:19:20 AM,Cor 3 Rojo,2019-03-25 06:19:20,6,Monday,3,3,1,0,0,1,44,1,0.727273,0.977273,0.022727


In [16]:
##PORCENTAJE DE USO POR CORREDOR##
df1 = pd.get_dummies(df1, columns=['CORREDOR'])

In [17]:
## LOOP PARA QUE AGRUPE LOS VAIAJES POR CORREDOR Y POR TARJETA
for column in df1.columns[-10:]:
    df1[f'{column}_suma'] = df1.groupby('TARJETA')[f'{column}'].transform('sum')


In [18]:
## AHORA DEBO CALCULAR LOS 8 PORCENTAJES
df1['porc_CORREDOR_Cor_1_Naranja']=df1['CORREDOR_Cor 1 Naranja_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_2_Azul']=df1['CORREDOR_Cor 2 Azul_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_3_Rojo']=df1['CORREDOR_Cor 3 Rojo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_4_Amarillo']=df1['CORREDOR_Cor 4 Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_5_Amarillo']=df1['CORREDOR_Cor 5 Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_6_Naranja']=df1['CORREDOR_Cor 6 Naranja_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_7_Azul']=df1['CORREDOR_Cor 7 Azul_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_8_Rojo']=df1['CORREDOR_Cor 8 Rojo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Lín_Anu_Amarillo']=df1['CORREDOR_Lín Anu Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_TROLE']=df1['CORREDOR_TROLE_suma']/ df1.VIAJES


In [19]:
##DEJAMOS SOLO LAS VARIABLES RELEVANTES PARA HACER EL ANALISIS DE CLUSTERS##
df2=df1.loc[:, ['TARJETA', 'VIAJES','porc_pico', 'porc_LaV', 'porc_Sab_Dom','porc_CORREDOR_Cor_1_Naranja', 'porc_CORREDOR_Cor_2_Azul','porc_CORREDOR_Cor_3_Rojo', 'porc_CORREDOR_Cor_4_Amarillo', 'porc_CORREDOR_Cor_5_Amarillo', 'porc_CORREDOR_Cor_6_Naranja','porc_CORREDOR_Cor_7_Azul', 'porc_CORREDOR_Cor_8_Rojo','porc_CORREDOR_Lín_Anu_Amarillo', 'porc_CORREDOR_TROLE', 'cant_mes_t1']]

In [ ]:
df2.head(20)

,TARJETA,VIAJES,porc_pico,porc_LaV,porc_Sab_Dom,porc_CORREDOR_Cor_1_Naranja,porc_CORREDOR_Cor_2_Azul,porc_CORREDOR_Cor_3_Rojo,porc_CORREDOR_Cor_4_Amarillo,porc_CORREDOR_Cor_5_Amarillo,porc_CORREDOR_Cor_6_Naranja,porc_CORREDOR_Cor_7_Azul,porc_CORREDOR_Cor_8_Rojo,porc_CORREDOR_Lín_Anu_Amarillo,porc_CORREDOR_TROLE,MES,cant_meses
0,2016806,44,0.727273,0.977273,0.022727,0.0,0.0,0.318182,0.0,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,3,3
1,2016806,44,0.727273,0.977273,0.022727,0.0,0.0,0.318182,0.0,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,1,3
2,2016806,44,0.727273,0.977273,0.022727,0.0,0.0,0.318182,0.0,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,3,3
3,2016806,44,0.727273,0.977273,0.022727,0.0,0.0,0.318182,0.0,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,3,3
4,2016806,44,0.727273,0.977273,0.022727,0.0,0.0,0.318182,0.0,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,3,3
5,2016806,44,0.727273,0.977273,0.022727,0.0,0.0,0.318182,0.0,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,3,3
6,2016806,44,0.727273,0.977273,0.022727,0.0,0.0,0.318182,0.0,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,3,3
7,2016806,44,0.727273,0.977273,0.022727,0.0,0.0,0.318182,0.0,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,2,3
8,2016806,44,0.727273,0.977273,0.022727,0.0,0.0,0.318182,0.0,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,1,3
9,2016806,44,0.727273,0.977273,0.022727,0.0,0.0,0.318182,0.0,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,1,3


In [20]:
df2=df2.drop_duplicates(subset='TARJETA')

In [ ]:
##OBTENGO MEDIDA RELATIVA DE INTENSIDAD DE USO##
df2.VIAJES.describe()

count    111412.000000
mean         84.972157
std          54.317490
min           1.000000
25%          46.000000
50%          78.000000
75%         114.000000
max         623.000000
Name: VIAJES, dtype: float64

In [ ]:
df2.head()

,TARJETA,VIAJES,porc_pico,porc_LaV,porc_Sab_Dom,porc_CORREDOR_Cor_1_Naranja,porc_CORREDOR_Cor_2_Azul,porc_CORREDOR_Cor_3_Rojo,porc_CORREDOR_Cor_4_Amarillo,porc_CORREDOR_Cor_5_Amarillo,porc_CORREDOR_Cor_6_Naranja,porc_CORREDOR_Cor_7_Azul,porc_CORREDOR_Cor_8_Rojo,porc_CORREDOR_Lín_Anu_Amarillo,porc_CORREDOR_TROLE,cant_meses
0,2016806,44,0.727273,0.977273,0.022727,0.000000,0.000000,0.318182,0.00000,0.181818,0.386364,0.022727,0.022727,0.0,0.068182,3
44,2020218,114,0.140351,0.640351,0.359649,0.043860,0.000000,0.000000,0.22807,0.061404,0.649123,0.000000,0.000000,0.0,0.017544,3
158,2020455,118,0.881356,0.983051,0.016949,0.898305,0.016949,0.016949,0.00000,0.025424,0.008475,0.000000,0.033898,0.0,0.000000,3
276,2020507,121,0.404959,0.925620,0.074380,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,3
397,2020753,47,0.829787,1.000000,0.000000,0.510638,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.489362,0.0,0.000000,3


In [21]:
para_a_nosup= df2
para_a_nosup.to_csv("para_a_nosup_1t19.csv", index=None)
from google.colab import files
files.download('para_a_nosup_1t19.csv') 



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
##LO MISMO PARA EL SEGUNDO TRIMESTRE DE 2019#
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
path= '/content/drive/My Drive/Mentoria/segundo trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()




Mounted at /content/drive/


,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO
0,05/04/2019 12:00:08 AM,04/04/2019 10:02:49 PM,4884082,Cor 8 Rojo,L82,Vuelta
1,05/04/2019 12:00:10 AM,04/04/2019 11:30:03 PM,5871414,TROLE,B,Vuelta
2,05/04/2019 12:00:12 AM,04/04/2019 10:43:21 PM,5813430,Cor 7 Azul,L75,Vuelta
3,05/04/2019 12:00:13 AM,04/04/2019 11:42:13 PM,3262031,Cor 2 Azul,L26,Ida
4,05/04/2019 12:00:15 AM,04/04/2019 10:59:19 PM,5791090,Cor 7 Azul,L74,Vuelta


In [2]:
import datetime as dt

In [3]:
# VARIABLE FECHA DE STRING A DATETIME##
big_frame['Fecha2'] = pd.to_datetime(big_frame['FECHA'], format='%d/%m/%Y %I:%M:%S %p')

In [4]:
##OBTENEMOS HORA DEL DÍA##
big_frame['hora'] = big_frame['Fecha2'].dt.hour

In [5]:
## OBTENEMOS DIA DE LA SEMANA##
big_frame['dia_semana']=big_frame['Fecha2'].dt.day_name()

In [6]:
## OBTENEMOS MES DEL AÑO###
big_frame['MES']=big_frame['Fecha2'].dt.month

In [7]:
big_frame['cant_mes_t2'] = big_frame.groupby('TARJETA')['MES'].transform('nunique')

In [ ]:
big_frame.head(3)

,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO,Fecha2,hora,dia_semana
0,05/04/2019 12:00:08 AM,04/04/2019 10:02:49 PM,4884082,Cor 8 Rojo,L82,Vuelta,2019-04-05 00:00:08,0,Friday
1,05/04/2019 12:00:10 AM,04/04/2019 11:30:03 PM,5871414,TROLE,B,Vuelta,2019-04-05 00:00:10,0,Friday
2,05/04/2019 12:00:12 AM,04/04/2019 10:43:21 PM,5813430,Cor 7 Azul,L75,Vuelta,2019-04-05 00:00:12,0,Friday


In [8]:
##GENERAMOS UNA VARIABLE QUE INDIQUE: LUNES A VIERNES##
mask=((big_frame.dia_semana=="Monday") | (big_frame.dia_semana=="Tuesday") | (big_frame.dia_semana=="Wednesday")  | (big_frame.dia_semana=="Thursday")   | (big_frame.dia_semana=="Friday") )  
big_frame['LaV']=0
big_frame.loc[mask, 'LaV']=1


In [ ]:
big_frame.LaV.value_counts()

1    10867325
0     1739876
Name: LaV, dtype: int64

In [9]:
## GENERAMOS UNA VARIABLE QUE INDIQUE: SABADOS Y DOMINGOS##
mask=((big_frame.dia_semana=="Saturday") | (big_frame.dia_semana=="Sunday"))
big_frame['Sabados_Domingos']=0
big_frame.loc[mask, 'Sabados_Domingos']=1

In [ ]:
big_frame.Sabados_Domingos.value_counts()

0    10867325
1     1739876
Name: Sabados_Domingos, dtype: int64

In [10]:
## GENERAMOS UN INDICADOR DE HORA DE USO##
## PICO / NO PICO##
## 7 ,8, 17, 18, 13, 14 HORAS DE MAYOR USO##
mask=(((big_frame.hora==7)| (big_frame.hora==8) | (big_frame.hora==17) | (big_frame.hora==18) | (big_frame.hora==13) | (big_frame.hora==14)) & (big_frame.LaV==1))
big_frame['PICO']=0
big_frame.loc[mask, 'PICO']=1

In [ ]:
big_frame.PICO.value_counts()

0    7617028
1    4990173
Name: PICO, dtype: int64

In [ ]:
big_frame.head(4)

,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO,Fecha2,hora,dia_semana,LaV,Sabados_Domingos,PICO
0,05/04/2019 12:00:08 AM,04/04/2019 10:02:49 PM,4884082,Cor 8 Rojo,L82,Vuelta,2019-04-05 00:00:08,0,Friday,1,0,0
1,05/04/2019 12:00:10 AM,04/04/2019 11:30:03 PM,5871414,TROLE,B,Vuelta,2019-04-05 00:00:10,0,Friday,1,0,0
2,05/04/2019 12:00:12 AM,04/04/2019 10:43:21 PM,5813430,Cor 7 Azul,L75,Vuelta,2019-04-05 00:00:12,0,Friday,1,0,0
3,05/04/2019 12:00:13 AM,04/04/2019 11:42:13 PM,3262031,Cor 2 Azul,L26,Ida,2019-04-05 00:00:13,0,Friday,1,0,0


In [ ]:
big_frame.CORREDOR.value_counts()

Cor 1 Naranja       2246655
Cor 6 Naranja       1945255
Cor 2 Azul          1746434
Cor 3 Rojo          1467839
Cor 7 Azul          1312638
Cor 5 Amarillo      1056606
Cor 4 Amarillo      1034857
Cor 8 Rojo          1008203
TROLE                598273
Lín Anu Amarillo     190441
Name: CORREDOR, dtype: int64

In [ ]:
## TOMARIA LAS SIGUIENTES VARIABLES PARA CONTAR POR TARJETA: %LaV %Sabados_Domingos %horaPICO %por CORREDOR
## OJO, ELIMINAR TARJETAS PRESTADAS ANTES
big_frame.shape


(12607201, 12)

In [11]:
#Se depura la base, nos quedamos solo con el primer uso de la tarjeta en ese mismo viaje.
df1 = big_frame.groupby(['TARJETA','LINEA','SENTIDO','FECHAAPERTURA']).first().reset_index()

In [12]:
##PORCENTAJE DE USO POR HORA PICO NO PICO POR TARJETA##
df1['CUENTA']=1
df1['VIAJES']=df1.groupby('TARJETA')['CUENTA'].transform('sum')



In [13]:
df1['AUX']=df1.groupby('TARJETA')['PICO'].transform('sum')
df1['porc_pico']=df1.AUX/ df1.VIAJES
#df1.head(5)



In [14]:
##PORCENTAJE DE USO DE LUNES A VIERNES##
df1['AUX']=df1.groupby('TARJETA')['LaV'].transform('sum')
df1['porc_LaV']=df1.AUX/ df1.VIAJES
#df1.head(5)

In [15]:
##PORCENTAJE DE USO EN SABADOS Y DOMINGOS##
df1['AUX']=df1.groupby('TARJETA')['Sabados_Domingos'].transform('sum')
df1['porc_Sab_Dom']=df1.AUX/ df1.VIAJES
#df1.head(5)

In [16]:
##PORCENTAJE DE USO POR CORREDOR##
df1 = pd.get_dummies(df1, columns=['CORREDOR'])

In [17]:
## LOOP PARA QUE AGRUPE LOS VAIAJES POR CORREDOR Y POR TARJETA
for column in df1.columns[-10:]:
    df1[f'{column}_suma'] = df1.groupby('TARJETA')[f'{column}'].transform('sum')


In [18]:
## AHORA DEBO CALCULAR LOS 8 PORCENTAJES
df1['porc_CORREDOR_Cor_1_Naranja']=df1['CORREDOR_Cor 1 Naranja_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_2_Azul']=df1['CORREDOR_Cor 2 Azul_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_3_Rojo']=df1['CORREDOR_Cor 3 Rojo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_4_Amarillo']=df1['CORREDOR_Cor 4 Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_5_Amarillo']=df1['CORREDOR_Cor 5 Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_6_Naranja']=df1['CORREDOR_Cor 6 Naranja_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_7_Azul']=df1['CORREDOR_Cor 7 Azul_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_8_Rojo']=df1['CORREDOR_Cor 8 Rojo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Lín_Anu_Amarillo']=df1['CORREDOR_Lín Anu Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_TROLE']=df1['CORREDOR_TROLE_suma']/ df1.VIAJES


In [19]:
##DEJAMOS SOLO LAS VARIABLES RELEVANTES PARA HACER EL ANALISIS DE CLUSTERS##
df2=df1.loc[:, ['TARJETA', 'VIAJES','porc_pico', 'porc_LaV', 'porc_Sab_Dom','porc_CORREDOR_Cor_1_Naranja', 'porc_CORREDOR_Cor_2_Azul','porc_CORREDOR_Cor_3_Rojo', 'porc_CORREDOR_Cor_4_Amarillo', 'porc_CORREDOR_Cor_5_Amarillo', 'porc_CORREDOR_Cor_6_Naranja','porc_CORREDOR_Cor_7_Azul', 'porc_CORREDOR_Cor_8_Rojo','porc_CORREDOR_Lín_Anu_Amarillo', 'porc_CORREDOR_TROLE', 'cant_mes_t2']]

In [20]:
del df1

In [21]:
df2=df2.drop_duplicates(subset='TARJETA')

In [ ]:
##OBTENGO MEDIDA RELATIVA DE INTENSIDAD DE USO##
#df2.VIAJES.describe()

In [22]:
para_a_nosup= df2
para_a_nosup.to_csv("para_a_nosup_2t19.csv", index=None)
from google.colab import files
files.download('para_a_nosup_2t19.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
##LO MISMO PARA EL TERCER TRIMESTRE DE 2019#
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
path= '/content/drive/My Drive/Mentoria/tercer trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()

Mounted at /content/drive/


,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO
0,06/07/2019 12:00:07 AM,05/07/2019 11:28:07 PM,5099043,Cor 1 Naranja,L12,Ida
1,06/07/2019 12:00:29 AM,05/07/2019 11:32:24 PM,5515754,Cor 6 Naranja,L67,Ida
2,06/07/2019 12:00:29 AM,05/07/2019 11:05:36 PM,4971142,Cor 5 Amarillo,L50,Vuelta
3,06/07/2019 12:00:30 AM,05/07/2019 11:05:36 PM,4971142,Cor 5 Amarillo,L50,Vuelta
4,06/07/2019 12:00:33 AM,05/07/2019 11:05:36 PM,4971142,Cor 5 Amarillo,L50,Vuelta


In [2]:
import datetime as dt

In [3]:
# VARIABLE FECHA DE STRING A DATETIME##
big_frame['Fecha2'] = pd.to_datetime(big_frame['FECHA'], format='%d/%m/%Y %I:%M:%S %p')

In [4]:
##OBTENEMOS HORA DEL DÍA##
big_frame['hora'] = big_frame['Fecha2'].dt.hour

In [5]:
## OBTENEMOS DIA DE LA SEMANA##
big_frame['dia_semana']=big_frame['Fecha2'].dt.day_name()

In [6]:
## OBTENEMOS MES DEL AÑO###
big_frame['MES']=big_frame['Fecha2'].dt.month

In [7]:
big_frame['cant_mes_t3'] = big_frame.groupby('TARJETA')['MES'].transform('nunique')

In [8]:
big_frame.head(3)

,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO,Fecha2,hora,dia_semana,MES,cant_mes_t3
0,06/07/2019 12:00:07 AM,05/07/2019 11:28:07 PM,5099043,Cor 1 Naranja,L12,Ida,2019-07-06 00:00:07,0,Saturday,7,3
1,06/07/2019 12:00:29 AM,05/07/2019 11:32:24 PM,5515754,Cor 6 Naranja,L67,Ida,2019-07-06 00:00:29,0,Saturday,7,3
2,06/07/2019 12:00:29 AM,05/07/2019 11:05:36 PM,4971142,Cor 5 Amarillo,L50,Vuelta,2019-07-06 00:00:29,0,Saturday,7,3


In [8]:
##GENERAMOS UNA VARIABLE QUE INDIQUE: LUNES A VIERNES##
mask=((big_frame.dia_semana=="Monday") | (big_frame.dia_semana=="Tuesday") | (big_frame.dia_semana=="Wednesday")  | (big_frame.dia_semana=="Thursday")   | (big_frame.dia_semana=="Friday") )  
big_frame['LaV']=0
big_frame.loc[mask, 'LaV']=1


In [ ]:
big_frame.LaV.value_counts()

In [9]:
## GENERAMOS UNA VARIABLE QUE INDIQUE: SABADOS Y DOMINGOS##
mask=((big_frame.dia_semana=="Saturday") | (big_frame.dia_semana=="Sunday"))
big_frame['Sabados_Domingos']=0
big_frame.loc[mask, 'Sabados_Domingos']=1

In [ ]:
big_frame.Sabados_Domingos.value_counts()

In [10]:
## GENERAMOS UN INDICADOR DE HORA DE USO##
## PICO / NO PICO##
## 7 ,8, 17, 18, 13, 14 HORAS DE MAYOR USO##
mask=(((big_frame.hora==7)| (big_frame.hora==8) | (big_frame.hora==17) | (big_frame.hora==18) | (big_frame.hora==13) | (big_frame.hora==14)) & (big_frame.LaV==1))
big_frame['PICO']=0
big_frame.loc[mask, 'PICO']=1

In [ ]:
big_frame.PICO.value_counts()

In [ ]:
big_frame.head(4)

In [ ]:
big_frame.CORREDOR.value_counts()

In [ ]:
## TOMARIA LAS SIGUIENTES VARIABLES PARA CONTAR POR TARJETA: %LaV %Sabados_Domingos %horaPICO %por CORREDOR
## OJO, ELIMINAR TARJETAS PRESTADAS ANTES
big_frame.shape


In [11]:
#Se depura la base, nos quedamos solo con el primer uso de la tarjeta en ese mismo viaje.
df1 = big_frame.groupby(['TARJETA','LINEA','SENTIDO','FECHAAPERTURA']).first().reset_index()

In [12]:
##PORCENTAJE DE USO POR HORA PICO NO PICO POR TARJETA##
df1['CUENTA']=1
df1['VIAJES']=df1.groupby('TARJETA')['CUENTA'].transform('sum')



In [13]:
df1['AUX']=df1.groupby('TARJETA')['PICO'].transform('sum')
df1['porc_pico']=df1.AUX/ df1.VIAJES
#df1.head(5)



In [14]:
##PORCENTAJE DE USO DE LUNES A VIERNES##
df1['AUX']=df1.groupby('TARJETA')['LaV'].transform('sum')
df1['porc_LaV']=df1.AUX/ df1.VIAJES
#df1.head(5)

In [15]:
##PORCENTAJE DE USO EN SABADOS Y DOMINGOS##
df1['AUX']=df1.groupby('TARJETA')['Sabados_Domingos'].transform('sum')
df1['porc_Sab_Dom']=df1.AUX/ df1.VIAJES
#df1.head(5)

In [16]:
##PORCENTAJE DE USO POR CORREDOR##
df1 = pd.get_dummies(df1, columns=['CORREDOR'])

In [17]:
## LOOP PARA QUE AGRUPE LOS VAIAJES POR CORREDOR Y POR TARJETA
for column in df1.columns[-10:]:
    df1[f'{column}_suma'] = df1.groupby('TARJETA')[f'{column}'].transform('sum')


In [18]:
## AHORA DEBO CALCULAR LOS 8 PORCENTAJES
df1['porc_CORREDOR_Cor_1_Naranja']=df1['CORREDOR_Cor 1 Naranja_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_2_Azul']=df1['CORREDOR_Cor 2 Azul_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_3_Rojo']=df1['CORREDOR_Cor 3 Rojo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_4_Amarillo']=df1['CORREDOR_Cor 4 Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_5_Amarillo']=df1['CORREDOR_Cor 5 Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_6_Naranja']=df1['CORREDOR_Cor 6 Naranja_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_7_Azul']=df1['CORREDOR_Cor 7 Azul_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_8_Rojo']=df1['CORREDOR_Cor 8 Rojo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Lín_Anu_Amarillo']=df1['CORREDOR_Lín Anu Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_TROLE']=df1['CORREDOR_TROLE_suma']/ df1.VIAJES


In [19]:
##DEJAMOS SOLO LAS VARIABLES RELEVANTES PARA HACER EL ANALISIS DE CLUSTERS##
df2=df1.loc[:, ['TARJETA', 'VIAJES','porc_pico', 'porc_LaV', 'porc_Sab_Dom','porc_CORREDOR_Cor_1_Naranja', 'porc_CORREDOR_Cor_2_Azul','porc_CORREDOR_Cor_3_Rojo', 'porc_CORREDOR_Cor_4_Amarillo', 'porc_CORREDOR_Cor_5_Amarillo', 'porc_CORREDOR_Cor_6_Naranja','porc_CORREDOR_Cor_7_Azul', 'porc_CORREDOR_Cor_8_Rojo','porc_CORREDOR_Lín_Anu_Amarillo', 'porc_CORREDOR_TROLE', 'cant_mes_t3']]

In [ ]:
#df2.columns

In [20]:
df2=df2.drop_duplicates(subset='TARJETA')

In [ ]:
##OBTENGO MEDIDA RELATIVA DE INTENSIDAD DE USO##
df2.VIAJES.describe()

In [21]:
para_a_nosup= df2
para_a_nosup.to_csv("para_a_nosup_3t19.csv", index=None)
from google.colab import files
files.download('para_a_nosup_3t19.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
##LO MISMO PARA EL CUARTO TRIMESTRE DE 2019#
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
path= '/content/drive/My Drive/Mentoria/cuarto trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()

Mounted at /content/drive/


,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO
0,06/10/2019 12:00:05 AM,05/10/2019 11:02:36 PM,5969486,Cor 7 Azul,L75,Vuelta
1,06/10/2019 12:00:09 AM,05/10/2019 11:26:36 PM,5943292,Cor 1 Naranja,L13,Ida
2,06/10/2019 12:00:13 AM,05/10/2019 11:46:15 PM,3497303,Cor 3 Rojo,L33,Ida
3,06/10/2019 12:00:15 AM,05/10/2019 11:46:15 PM,4659858,Cor 3 Rojo,L33,Ida
4,06/10/2019 12:00:18 AM,05/10/2019 10:27:52 PM,5325561,Cor 6 Naranja,L60,Vuelta


In [4]:
import datetime as dt

In [5]:
# VARIABLE FECHA DE STRING A DATETIME##
big_frame['Fecha2'] = pd.to_datetime(big_frame['FECHA'], format='%d/%m/%Y %I:%M:%S %p')

In [6]:
##OBTENEMOS HORA DEL DÍA##
big_frame['hora'] = big_frame['Fecha2'].dt.hour

In [7]:
## OBTENEMOS DIA DE LA SEMANA##
big_frame['dia_semana']=big_frame['Fecha2'].dt.day_name()

In [8]:
## OBTENEMOS MES DEL AÑO###
big_frame['MES']=big_frame['Fecha2'].dt.month

In [9]:
big_frame['cant_mes_t4'] = big_frame.groupby('TARJETA')['MES'].transform('nunique')

In [ ]:
big_frame.head(3)

In [10]:
##GENERAMOS UNA VARIABLE QUE INDIQUE: LUNES A VIERNES##
mask=((big_frame.dia_semana=="Monday") | (big_frame.dia_semana=="Tuesday") | (big_frame.dia_semana=="Wednesday")  | (big_frame.dia_semana=="Thursday")   | (big_frame.dia_semana=="Friday") )  
big_frame['LaV']=0
big_frame.loc[mask, 'LaV']=1


In [ ]:
big_frame.LaV.value_counts()

In [11]:
## GENERAMOS UNA VARIABLE QUE INDIQUE: SABADOS Y DOMINGOS##
mask=((big_frame.dia_semana=="Saturday") | (big_frame.dia_semana=="Sunday"))
big_frame['Sabados_Domingos']=0
big_frame.loc[mask, 'Sabados_Domingos']=1

In [ ]:
big_frame.Sabados_Domingos.value_counts()

0    9694457
1    1500185
Name: Sabados_Domingos, dtype: int64

In [12]:
## GENERAMOS UN INDICADOR DE HORA DE USO##
## PICO / NO PICO##
## 7 ,8, 17, 18, 13, 14 HORAS DE MAYOR USO##
mask=(((big_frame.hora==7)| (big_frame.hora==8) | (big_frame.hora==17) | (big_frame.hora==18) | (big_frame.hora==13) | (big_frame.hora==14)) & (big_frame.LaV==1))
big_frame['PICO']=0
big_frame.loc[mask, 'PICO']=1

In [ ]:
big_frame.PICO.value_counts()

In [ ]:
big_frame.head(4)

In [ ]:
big_frame.CORREDOR.value_counts()

In [ ]:
## TOMARIA LAS SIGUIENTES VARIABLES PARA CONTAR POR TARJETA: %LaV %Sabados_Domingos %horaPICO %por CORREDOR
## OJO, ELIMINAR TARJETAS PRESTADAS ANTES
big_frame.shape


In [13]:
#Se depura la base, nos quedamos solo con el primer uso de la tarjeta en ese mismo viaje.
df1 = big_frame.groupby(['TARJETA','LINEA','SENTIDO','FECHAAPERTURA']).first().reset_index()

In [14]:
##PORCENTAJE DE USO POR HORA PICO NO PICO POR TARJETA##
df1['CUENTA']=1
df1['VIAJES']=df1.groupby('TARJETA')['CUENTA'].transform('sum')



In [15]:
df1['AUX']=df1.groupby('TARJETA')['PICO'].transform('sum')
df1['porc_pico']=df1.AUX/ df1.VIAJES
#df1.head(5)



In [16]:
##PORCENTAJE DE USO DE LUNES A VIERNES##
df1['AUX']=df1.groupby('TARJETA')['LaV'].transform('sum')
df1['porc_LaV']=df1.AUX/ df1.VIAJES
#df1.head(5)

In [17]:
##PORCENTAJE DE USO EN SABADOS Y DOMINGOS##
df1['AUX']=df1.groupby('TARJETA')['Sabados_Domingos'].transform('sum')
df1['porc_Sab_Dom']=df1.AUX/ df1.VIAJES
#df1.head(5)

In [18]:
##PORCENTAJE DE USO POR CORREDOR##
df1 = pd.get_dummies(df1, columns=['CORREDOR'])

In [19]:
## LOOP PARA QUE AGRUPE LOS VAIAJES POR CORREDOR Y POR TARJETA
for column in df1.columns[-10:]:
    df1[f'{column}_suma'] = df1.groupby('TARJETA')[f'{column}'].transform('sum')


In [20]:
## AHORA DEBO CALCULAR LOS 8 PORCENTAJES
df1['porc_CORREDOR_Cor_1_Naranja']=df1['CORREDOR_Cor 1 Naranja_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_2_Azul']=df1['CORREDOR_Cor 2 Azul_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_3_Rojo']=df1['CORREDOR_Cor 3 Rojo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_4_Amarillo']=df1['CORREDOR_Cor 4 Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_5_Amarillo']=df1['CORREDOR_Cor 5 Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_6_Naranja']=df1['CORREDOR_Cor 6 Naranja_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_7_Azul']=df1['CORREDOR_Cor 7 Azul_suma']/ df1.VIAJES
df1['porc_CORREDOR_Cor_8_Rojo']=df1['CORREDOR_Cor 8 Rojo_suma']/ df1.VIAJES
df1['porc_CORREDOR_Lín_Anu_Amarillo']=df1['CORREDOR_Lín Anu Amarillo_suma']/ df1.VIAJES
df1['porc_CORREDOR_TROLE']=df1['CORREDOR_TROLE_suma']/ df1.VIAJES


In [21]:
##DEJAMOS SOLO LAS VARIABLES RELEVANTES PARA HACER EL ANALISIS DE CLUSTERS##
df2=df1.loc[:, ['TARJETA', 'VIAJES','porc_pico', 'porc_LaV', 'porc_Sab_Dom','porc_CORREDOR_Cor_1_Naranja', 'porc_CORREDOR_Cor_2_Azul','porc_CORREDOR_Cor_3_Rojo', 'porc_CORREDOR_Cor_4_Amarillo', 'porc_CORREDOR_Cor_5_Amarillo', 'porc_CORREDOR_Cor_6_Naranja','porc_CORREDOR_Cor_7_Azul', 'porc_CORREDOR_Cor_8_Rojo','porc_CORREDOR_Lín_Anu_Amarillo', 'porc_CORREDOR_TROLE', 'cant_mes_t4']]

In [ ]:
df2.columns

In [22]:
df2=df2.drop_duplicates(subset='TARJETA')

In [ ]:
##OBTENGO MEDIDA RELATIVA DE INTENSIDAD DE USO##
df2.VIAJES.describe()

In [23]:
para_a_nosup= df2
para_a_nosup.to_csv("para_a_nosup_4t19.csv", index=None)
from google.colab import files
files.download('para_a_nosup_4t19.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>